In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)

In [ ]:
# %%
# WWF ANIMAL DATA

# read in all sheets of ods excel file for animal data
df = pd.read_excel('/home/muskrat/Documents/eco_data_copy/main_eco_data/animal_copy.ods',
                   index_col=0, sheet_name=None, engine=('odf'))


In [ ]:

# %%
# WWF ANIMAL DATA

# turn each sheet into dataframe
ecoregions = df['ecoregions']
ecoregions.reset_index(inplace=True)
classes = df['class']
common_names = df['common_names']
common_names_bup = df['common_names_bup']
eco_species = df['ecoregion_species']
family = df['family']
genus = df['genus']
order = df['order_']
species = df['species']
# add unique id to each ecoregion and convert from int to str
ranger = [*range(1, 1 + len(ecoregions))]
rover = [str(x) for x in ranger]

ecoregions.insert(0, 'unique_id', rover)

# print(species.head())
# print(genus.head())

# df1 = common_names_bup.merge(common_names, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='left_only']
# df1


In [ ]:

# %%
# WWF ANIMAL DATA

# join species and common name dataframes on species id
species_common = pd.merge(species, common_names_bup, on='SPECIES_ID', how='left').reindex(
    columns=['SPECIES_ID', 'SPECIES', 'COMMON_NAME', 'GENUS_ID'])
# find missing value index
print(species_common.loc[pd.isna(species_common["SPECIES"]), :].index)

# print(species_common.head(10))

genus_species = pd.merge(species_common, genus, on='GENUS_ID', how='left').reindex(
    columns=['SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(genus_species.loc[pd.isna(genus_species["GENUS"]), :].index)

family_genus = pd.merge(genus_species, family, on='FAMILY_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(family_genus.loc[pd.isna(family_genus["FAMILY"]), :].index)

order_family = pd.merge(family_genus, order, on='ORDER_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(order_family.loc[pd.isna(order_family["ORDER_DESC"]), :].index)

class_order = pd.merge(order_family, classes, on='CLASS_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'COMMON_NAME'])
print(class_order.loc[pd.isna(class_order["CLASS"]), :].index)



In [ ]:

# %%
# WWF ANIMAL DATA

# add genus and species columns into one scientific name column
class_order['Scientific_Name'] = class_order['GENUS'].str.cat(
    class_order['SPECIES'], sep=" ")
# print(class_order.loc[pd.isna(class_order["Scientific_Name"]), :].index)

# find all missing values
# missing = class_order[class_order.isna().any(axis=1)]

# fill missing values with unknown
# class_order.fillna('Unknown', inplace=True)

eco_mod = pd.merge(class_order, eco_species, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE'])

eco_mod_miss = eco_mod.loc[pd.isna(eco_mod["ECOREGION_CODE"]), :].index


ecos = pd.merge(eco_mod, ecoregions, on='ECOREGION_CODE', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'ECOREGION_NAME', 'unique_id'])

ecos.dropna(subset=['unique_id'], inplace=True)
# %%
# print(ecos.loc[pd.isna(ecos["ECOREGION_NAME"]), :].index)

# ecos.fillna('Unknown', inplace=True)

# create a list of all ecoregion names that apply to same species id
ecos1 = ecos.groupby('SPECIES_ID')['ECOREGION_NAME'].apply(list).reset_index()


ecos2 = ecos.groupby('SPECIES_ID')['ECOREGION_CODE'].apply(list).reset_index()
# print(ecos1.head(15))

ecos3 = ecos.groupby('SPECIES_ID')['unique_id'].apply(list).reset_index()

ecos4 = pd.merge(class_order, ecos1, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_NAME'])

ecos5 = pd.merge(ecos4, ecos2, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'ECOREGION_NAME'])

ecos6 = pd.merge(ecos5, ecos3, on='SPECIES_ID', how='left').reindex(columns=[
    'SPECIES_ID', 'GENUS_ID', 'FAMILY_ID', 'ORDER_ID', 'CLASS_ID', 'CLASS', 'ORDER_DESC', 'FAMILY', 'GENUS', 'SPECIES', 'Scientific_Name', 'COMMON_NAME', 'ECOREGION_CODE', 'unique_id', 'ECOREGION_NAME'])

final = ecos6[['SPECIES_ID', 'ORDER_DESC', 'FAMILY', 'CLASS', 'Scientific_Name',
               'COMMON_NAME', 'ECOREGION_CODE', 'unique_id', 'ECOREGION_NAME']]


In [ ]:

# %%

# create dataframe of all values in class column that are mammals
mammals = final.loc[final['CLASS'].isin(['Mammalia'])].reset_index(drop=True)

amphibians = final.loc[final['CLASS'].isin(
    ['Amphibia'])].reset_index(drop=True)

reptiles = final.loc[final['CLASS'].isin(['Reptilia'])].reset_index(drop=True)

birds = final.loc[final['CLASS'].isin(['Aves'])].reset_index(drop=True)

# create dataframe of all missing values
unknown = final.loc[final['CLASS'].isna()]

# create dataframe of all missing common names
common_miss = final.loc[final['COMMON_NAME'].isna()]

final.drop(final.index[[27729, 27779, 27971]], inplace=True,)
final.reset_index(drop=True, inplace=True)


In [ ]:

# %%

# convert to json
mammalj = mammals.to_json(orient='records', force_ascii=False)
repr(mammalj)

amphibianj = amphibians.to_json(orient='records', force_ascii=False)
repr(amphibianj)

reptilej = reptiles.to_json(orient='records', force_ascii=False)
repr(reptilej)

birdj = birds.to_json(orient='records', force_ascii=False)
repr(birdj)

finalj = final.to_json(orient='records', force_ascii=False)
repr(finalj)

final_miss = common_miss.to_json(orient='records', force_ascii=False)
repr(final_miss)


In [ ]:

# %%
# WWF ANIMAL DATA

# write to files
# file = open("wwf_mammal.json", "w")
# file.write(mammalj)
# file.close

# file = open("wwf_amphibian.json", "w")
# file.write(amphibianj)
# file.close

# file = open("wwf_reptile.json", "w")
# file.write(reptilej)
# file.close

# file = open("birds_string.json", "w")
# file.write(birdj)
# file.close

file = open("wwf_animal_full.json", "w")
file.write(finalj)
file.close

file = open("wwf_animal_full_miss.json", "w")
file.write(final_miss)
file.close
